In [1]:
# Install dependencies
!pip3.9 install --upgrade pip
!pip3.9 install pandas tqdm multiprocess

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [2]:
# from tqdm.notebook import tqdm, trange
from tqdm import tqdm
import pandas as pd
import numpy as np
import glob

In [3]:
datadir = "./data/*/"
all_datas = sorted(glob.glob(datadir + "*.csv"))

In [4]:
# CSV Count
print("Data CSV Count:", len(all_datas))

Data CSV Count: 39364


In [5]:
from multiprocess import Process, Pool

print("Read files...")

def read(filename):
    name = filename
    
    data = pd.read_csv(filename)
    data["filename"] = name
    # print("Read", filename.split("/")[-1])
    return (name, data)

pool = Pool()
result = list(tqdm(pool.imap(read, all_datas), total=len(all_datas)))

pool.close()
pool.join() # wait till pool to finish calculation

print("Finished Reading files...")

all_df = dict(result)

Read files...


100%|███████████████████████████████████| 39364/39364 [00:35<00:00, 1121.01it/s]

Finished Reading files...


In [6]:
# 버스 정류장
busstation1 = "PHB351016075" # 조회할 버스 정류장 - PHB351016075 농협양덕지점 (한동대방향)
busstation2 = "PHB351016076" # 조회할 버스 정류장2 (지나갔을까봐) - PHB351016075 농협양덕지점 (한동대방향)

all_datas = pd.concat(all_df.values(), ignore_index=True) # 모든 데이터를 합친다
all_datas = all_datas[(all_datas.nodeid == busstation1) | (all_datas.nodeid == busstation2)] # 정류장 검색

all_datas.reset_index(inplace=True) # Reset index
all_datas = all_datas[["vehicleno", "nodenm", "filename"]] # vehicle 번호와 filename 만 보여주기
all_datas["time"] = pd.to_datetime(all_datas["filename"], format="%y%m%d/%H%M%S", errors="coerce", exact=False) # filename 에서 시간 계산
all_datas.set_index("time", inplace=True)

all_datas.head()

,vehicleno,nodenm,filename
time,,,
2021-08-25 09:09:02,1431,남광하우스토리,./data/210825/090902.csv
2021-08-25 09:10:03,1431,남광하우스토리,./data/210825/091003.csv
2021-08-25 09:22:02,1383,농협 양덕지점,./data/210825/092202.csv
2021-08-25 09:23:03,1383,농협 양덕지점,./data/210825/092303.csv
2021-08-25 09:24:03,1383,농협 양덕지점,./data/210825/092403.csv


In [7]:
# 다음 entry 가 같은 vehicleno 일경우 무시
all_datas = all_datas[ 
    all_datas.vehicleno.ne(all_datas.vehicleno.shift())
]
all_datas.head()

,vehicleno,nodenm,filename
time,,,
2021-08-25 09:09:02,1431,남광하우스토리,./data/210825/090902.csv
2021-08-25 09:22:02,1383,농협 양덕지점,./data/210825/092202.csv
2021-08-25 09:39:02,1323,농협 양덕지점,./data/210825/093902.csv
2021-08-25 10:01:03,1349,농협 양덕지점,./data/210825/100103.csv
2021-08-25 10:18:03,1348,농협 양덕지점,./data/210825/101803.csv


In [8]:
# 빈파일 목록 확인
empty_files = []
for name, data in all_df.items():
    if len(data) == 0:
        empty_files.append(name)
len(empty_files)

331

In [38]:
# 빈파일들에는 null 데이터를 추가하여 다음 시간이 계산이 안되게 처리.
for file in empty_files:
    datetime = pd.to_datetime(file, format="%y%m%d/%H%M%S", errors="coerce", exact=False)
    all_datas.loc[datetime] = [np.nan for i in range(len(all_datas.columns))]

In [39]:
# 마지막으로 전에 빈 공간이 있을경우 데이터 무시.
all_datas = all_datas.sort_index()
all_datas["diff"] = all_datas.index.to_series().diff()
all_datas = all_datas[~all_datas.vehicleno.shift().isna()].dropna(subset=['filename'])

In [35]:
# 시간 계산
# all_datas["diff"] = all_datas.index.to_series().diff()

In [40]:
# all_datas["diff"]
diff_data = all_datas.dropna(subset=["diff"])["diff"].dt.seconds.div(60, fill_value=0)
diff_data[(diff_data < 120) & (diff_data > 2)].describe() # 최대시간을 2시간으로 두어 익일인 경우 무시.

count    2089.000000
mean       18.067321
std         6.504762
min         2.966667
25%        14.000000
50%        18.000000
75%        21.983333
max        54.000000
Name: diff, dtype: float64

------------

# Test Process

In [43]:
copy = pd.DataFrame(all_datas)
copy.head()

,vehicleno,nodenm,filename,diff
time,,,,
2021-08-25 10:39:02,1428,농협 양덕지점,./data/210825/103902.csv,0 days 00:20:59
2021-08-25 11:03:03,1346,농협 양덕지점,./data/210825/110303.csv,0 days 00:24:01
2021-08-25 11:16:03,1377,농협 양덕지점,./data/210825/111603.csv,0 days 00:13:00
2021-08-25 11:30:03,1236,농협 양덕지점,./data/210825/113003.csv,0 days 00:14:00
2021-08-25 11:47:02,1338,농협 양덕지점,./data/210825/114702.csv,0 days 00:16:59


In [14]:
copy.dropna(subset=["diff"], inplace=True)
copy['dval'] = all_datas.dropna(subset=["diff"])["diff"].dt.seconds.div(60, fill_value=0)
copy.head()

,vehicleno,nodenm,filename,diff,dval
time,,,,,
2021-08-25 09:39:02,1323,농협 양덕지점,./data/210825/093902.csv,0 days 00:17:00,17.000000
2021-08-25 10:01:03,1349,농협 양덕지점,./data/210825/100103.csv,0 days 00:22:01,22.016667
2021-08-25 10:18:03,1348,농협 양덕지점,./data/210825/101803.csv,0 days 00:17:00,17.000000
2021-08-25 10:39:02,1428,농협 양덕지점,./data/210825/103902.csv,0 days 00:20:59,20.983333
2021-08-25 11:03:03,1346,농협 양덕지점,./data/210825/110303.csv,0 days 00:24:01,24.016667


In [24]:
copy.loc[copy['dval'] > 70].sort_values(by=["dval"]).head()

,vehicleno,nodenm,filename,diff,dval
time,,,,,
2021-09-14 20:20:04,1377,농협 양덕지점,./data/210914/202004.csv,0 days 01:22:01,82.016667
2021-09-15 12:36:05,1338,농협 양덕지점,./data/210915/123605.csv,0 days 02:01:02,121.033333
2021-09-15 17:02:03,1428,농협 양덕지점,./data/210915/170203.csv,0 days 02:36:00,156.000000
2021-09-14 16:17:03,1236,농협 양덕지점,./data/210914/161703.csv,0 days 03:03:00,183.000000
2021-09-18 09:00:04,1236,농협 양덕지점,./data/210918/090004.csv,0 days 10:01:01,601.016667


In [16]:
copy.loc[copy['dval'] == 0]

,vehicleno,nodenm,filename,diff,dval
time,,,,,
2021-08-25 19:13:03,1348,남광하우스토리,./data/210825/191303.csv,0 days,0.0
2021-08-25 19:14:03,1348,남광하우스토리,./data/210825/191403.csv,0 days,0.0
2021-08-30 21:02:03,1377,남광하우스토리,./data/210830/210203.csv,0 days,0.0
2021-08-30 21:03:03,1377,남광하우스토리,./data/210830/210303.csv,0 days,0.0
2021-08-31 17:35:03,1338,남광하우스토리,./data/210831/173503.csv,0 days,0.0
...,...,...,...,...,...
2021-10-12 17:33:04,1405,남광하우스토리,./data/211012/173304.csv,0 days,0.0
2021-10-15 19:29:03,1346,남광하우스토리,./data/211015/192903.csv,0 days,0.0
2021-10-17 20:15:03,1377,남광하우스토리,./data/211017/201503.csv,0 days,0.0


In [17]:
# all Days
print("Number of days:", len(copy.groupby(pd.Grouper(freq='D'))))

Number of days: 55


In [41]:
copy.sort_values(['time', 'vehicleno']).loc["2021-10-18"]

,vehicleno,nodenm,filename,diff,dval
time,,,,,
2021-10-18 09:05:03,1261,농협 양덕지점,./data/211018/090503.csv,0 days 10:16:00,616.000000
2021-10-18 09:17:03,1382,농협 양덕지점,./data/211018/091703.csv,0 days 00:12:00,12.000000
2021-10-18 09:22:03,1382,남광하우스토리,./data/211018/092203.csv,0 days 00:05:00,5.000000
2021-10-18 09:22:03,1429,농협 양덕지점,./data/211018/092203.csv,0 days 00:00:00,0.000000
2021-10-18 09:23:03,1382,남광하우스토리,./data/211018/092303.csv,0 days 00:00:00,0.000000
2021-10-18 09:23:03,1429,농협 양덕지점,./data/211018/092303.csv,0 days 00:01:00,1.000000
2021-10-18 09:24:02,1429,농협 양덕지점,./data/211018/092402.csv,0 days 00:00:59,0.983333
2021-10-18 09:50:02,1239,농협 양덕지점,./data/211018/095002.csv,0 days 00:26:00,26.000000
2021-10-18 09:57:03,1431,농협 양덕지점,./data/211018/095703.csv,0 days 00:07:01,7.016667


In [19]:
# 빈파일 목록 확인
empty_files = []
for name, data in all_df.items():
    if len(data) == 0:
        empty_files.append(name)
len(empty_files)

331

In [20]:
import numpy as np
# 빈파일들에는 null 데이터를 추가하여 다음 시간이 계산이 안되게 처리.
for file in empty_files:
    datetime = pd.to_datetime(file, format="%y%m%d/%H%M%S", errors="coerce", exact=False)
    copy.loc[datetime] = [np.nan for i in range(len(copy.columns))]

In [21]:
copyr = copy.sort_index()
copyr["diff2"] = copyr.index.to_series().diff()
copyr = copyr[~copyr.vehicleno.shift().isna()].dropna(subset=['filename'])
print(len(copyr))
# copyr.loc['2021-09-14 13:30':'2021-09-14 16:30']

2295


In [22]:
# all_datas["diff"]
copyr_diffdata = copyr.dropna(subset=["diff"])["diff"].dt.seconds.div(60, fill_value=0)
copyr_diffdata[(copyr_diffdata < 120) & (copyr_diffdata > 2)].describe() # 최대시간을 2시간으로 두어 익일인 경우 무시.

count    2091.000000
mean       18.068699
std         6.502265
min         2.966667
25%        14.000000
50%        18.000000
75%        21.983333
max        54.000000
Name: diff, dtype: float64

In [23]:
# Remove diff data if last row doesn't have any information
for item in copyr.index:
    print(copyr.vehicleno.shift().loc[item])
    if pd.isnull(copyr.vehicleno.shift().loc[item]):
        copyr.loc[item, "dval"] = np.nan

nan
1349
1348
1428
1346
1377
1236
1338
1239
1343
1261
1382
1429
1405
1383
1229
1349
1348
1428
1346
1377
1236
1338
1239
1343
1261
1382
1429
1431
1383
1323
1349
time
2021-08-25 19:13:03    1348
2021-08-25 19:13:03    1428
Name: vehicleno, dtype: object


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
copyr.diff2

In [ ]:
copy

In [ ]:
copy.sort_index().loc['2021-09-14 13:30':'2021-09-14 16:30']

In [30]:
pd.set_option('display.max_rows', None)

In [44]:
copy.sort_index().loc['2021-09-14']

,vehicleno,nodenm,filename,diff
time,,,,
2021-09-14 09:04:03,1261,남광하우스토리,./data/210914/090403.csv,0 days 10:05:01
2021-09-14 09:13:03,1382,농협 양덕지점,./data/210914/091303.csv,0 days 00:09:00
2021-09-14 09:22:02,1429,농협 양덕지점,./data/210914/092202.csv,0 days 00:08:59
2021-09-14 09:48:02,1323,농협 양덕지점,./data/210914/094802.csv,0 days 00:26:00
2021-09-14 09:57:03,1431,농협 양덕지점,./data/210914/095703.csv,0 days 00:09:01
2021-09-14 10:16:04,1383,농협 양덕지점,./data/210914/101604.csv,0 days 00:19:01
2021-09-14 10:39:03,1349,농협 양덕지점,./data/210914/103903.csv,0 days 00:22:59
2021-09-14 10:58:03,1348,농협 양덕지점,./data/210914/105803.csv,0 days 00:19:00
2021-09-14 11:16:03,1428,농협 양덕지점,./data/210914/111603.csv,0 days 00:18:00


In [ ]:
# Filter all empty files
len(result)

In [ ]:
# https://stackoverflow.com/questions/56057755/extract-all-rows-in-dataframe-having-same-date
for name, group in copy.groupby(pd.Grouper(freq='D')):
    print(name)